<a href="https://colab.research.google.com/github/drewamorbordelon/DS-Unit-4-Sprint-1-NLP/blob/main/module2-vector-representations/LS_DS_412_Vector_Representations_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Vector Representations
## *Data Science Unit 4 Sprint 2 Assignment 2*

In [ ]:
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy

from bs4 import BeautifulSoup

## 1) *Clean:* Job Listings from indeed.com that contain the title "Data Scientist" 

You have `job_listings.csv` in the data folder for this module. The text data in the description column is still messy - full of html tags. Use the [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) library to clean up this column. You will need to read through the documentation to accomplish this task. 

In [ ]:
df = pd.read_csv('job_listings.csv')

In [ ]:
df['description'] = [BeautifulSoup(text).get_text() for text in df['description'] ]
df['description'] = df['description'].map(lambda x: x.lstrip("'b'"))
df['description'] = df['description'].map(lambda x: x.lstrip('"'))

In [ ]:

def capital_words_spaces(str1):
  return re.sub(r"(\w)([A-Z])", r"\1 \2", str1)
df['description'] = df['description'].apply(capital_words_spaces)

In [ ]:
from bs4 import BeautifulSoup
import requests

df['description'] = [BeautifulSoup(text).get_text() for text in df['description'] ]

#raise Exception("\nThis task is not complete. \nReplace this line with your code for the task.")
                


In [ ]:
df.head()

,Unnamed: 0,description,title
0,0,Job Requirements:\n Conceptual understanding i...,Data scientist
1,1,"Job Description\n\n As a Data Scientist 1, you...",Data Scientist I
2,2,As a Data Scientist you will be working on con...,Data Scientist - Entry Level
3,3,"$4,969 - $6,756 a month Contract Under the gen...",Data Scientist
4,4,Location: U SA \xe2\x80\x93 multiple locations...,Data Scientist


In [ ]:
df = df.drop(['Unnamed: 0'], axis = 1)

## 2) Use Spacy to tokenize the listings 

In [ ]:
import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_lg")

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [ ]:
tokens = []

""" Update those tokens w/o stopwords"""
for doc in tokenizer.pipe(df['description'], batch_size=500):
    
    doc_tokens = []
    doc_tokens = [token.lemma_ for token in doc if (token.is_stop == False) & (token.is_punct == False)]
    doc_tokens.append(token.text.lower())

    tokens.append(doc_tokens)

df['tokens'] = tokens

In [ ]:
def count(docs):

        word_counts = Counter()
        appears_in = Counter()
        
        total_docs = len(docs)

        for doc in docs:
            word_counts.update(doc)
            appears_in.update(set(doc))

        temp = zip(word_counts.keys(), word_counts.values())
        
        wc = pd.DataFrame(temp, columns = ['word', 'count'])

        wc['rank'] = wc['count'].rank(method='first', ascending=False)
        total = wc['count'].sum()

        wc['pct_total'] = wc['count'].apply(lambda x: x / total)
        
        wc = wc.sort_values(by='rank')
        wc['cul_pct_total'] = wc['pct_total'].cumsum()

        t2 = zip(appears_in.keys(), appears_in.values())
        ac = pd.DataFrame(t2, columns=['word', 'appears_in'])
        wc = ac.merge(wc, on='word')

        wc['appears_in_pct'] = wc['appears_in'].apply(lambda x: x / total_docs)
        
        return wc.sort_values(by='rank')

In [ ]:
wc = count(df['tokens'])
wc.head(20)

,word,appears_in,count,rank,pct_total,cul_pct_total,appears_in_pct
51,datum,412,2975,1.0,0.023695,0.023695,0.967136
20,work,361,1280,2.0,0.010195,0.033890,0.847418
225,team,339,988,3.0,0.007869,0.041760,0.795775
25,experience,365,982,4.0,0.007821,0.049581,0.856808
96,business,298,975,5.0,0.007766,0.057347,0.699531
302,Data,318,781,6.0,0.006221,0.063567,0.746479
7,model,281,767,7.0,0.006109,0.069676,0.659624
316,learn,284,666,8.0,0.005305,0.074981,0.666667
250,product,230,585,9.0,0.004659,0.079640,0.539906
154,build,278,574,10.0,0.004572,0.084212,0.652582


In [ ]:
df.head()

,description,title,tokens
0,Job Requirements:\nConceptual understanding in...,Data scientist,"[Job, Requirements:\nConceptual, understand, M..."
1,"Job Description\n\nAs a Data Scientist 1, you ...",Data Scientist I,"[Job, Description\n\nAs, Data, Scientist, 1,, ..."
2,As a Data Scientist you will be working on con...,Data Scientist - Entry Level,"[Data, Scientist, work, consult, business., re..."
3,"$4,969 - $6,756 a monthContractUnder the gener...",Data Scientist,"[$4,969, $6,756, monthContractUnder, general, ..."
4,Location: USA \xe2\x80\x93 multiple locations\...,Data Scientist,"[Location:, USA, \xe2\x80\x93, multiple, locat..."


In [ ]:
# def strip_end(text, suffix):
#     if not text.endswith(suffix):
#         return text
#     return text[:len(text)-len(suffix)]

## 3) Use Scikit-Learn's CountVectorizer to get word counts for each listing.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

In [ ]:
##### Your Code Here #####
text = df['description']
# create the transformer
#df = df.drop[cols for call if ]
#df = df.drop([cols for cols in df.columns if len(cols) <3])
#bad = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
vect = CountVectorizer(stop_words=['english'], max_features=5000)

# build vocab
vect.fit(text)

# transform text
dtm = vect.transform(text)

dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())

In [ ]:
dtm

,000,04,10,100,1079302,11,12,125,14,15,...,you,young,your,yours,yourself,yrs,zero,zeus,zf,zillow
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,2,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,0,0,0,0,0,0,0,0,0,0,...,3,0,1,0,0,0,0,0,0,0
422,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
423,2,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,1,0,0,0
424,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0


## 4) Visualize the most common word counts

In [ ]:
wc['word'].value_counts()

1    1
e    1
w    1
0    1
7    1
t    1
u    1
g    1
2    1
c    1
v    1
9    1
n    1
q    1
8    1
5    1
3    1
p    1
k    1
z    1
s    1
a    1
j    1
i    1
l    1
x    1
b    1
f    1
o    1
d    1
r    1
6    1
y    1
m    1
4    1
h    1
Name: word, dtype: int64

In [ ]:
##### Your Code Here #####
wc = count(dtm)
wc.head(20)

,word,appears_in,count,rank,pct_total,cul_pct_total,appears_in_pct
15,e,3092,4317,1.0,0.117275,0.117275,7.258216
22,n,2543,3309,2.0,0.089892,0.207166,5.969484
23,i,2459,3293,3.0,0.089457,0.296623,5.772300
13,t,2276,2795,4.0,0.075928,0.372552,5.342723
14,a,2263,2782,5.0,0.075575,0.448127,5.312207
11,s,2120,2654,6.0,0.072098,0.520225,4.976526
10,r,2151,2493,7.0,0.067724,0.587949,5.049296
25,o,1912,2290,8.0,0.062210,0.650159,4.488263
24,l,1515,1772,9.0,0.048138,0.698297,3.556338
28,c,1509,1713,10.0,0.046535,0.744832,3.542254


## 5) Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
# Create a vocabulary and get word counts per document
# Similiar to fit_predict
dtm = tfidf.fit_transform(data)
# Print word counts
# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
# View Feature Matrix as DataFrame
dtm.head()

,000,04,10,100,1079302,11,12,125,14,15,...,years,yearthe,yes,yeti,york,young,yrs,zeus,zf,zillow
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.093431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_ for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [ ]:
##### Your Code Here #####
# Tunning Parameters
# Instantiate vectorizer object
data = df['description']
tfidf = TfidfVectorizer(stop_words='english', 
                        ngram_range=(1,2),
                        max_df=.75,
                        min_df=30,
                        tokenizer=tokenize)
# Create a vocabulary and get word counts per document
dtm2 = tfidf.fit_transform(data) # Similiar to fit_predict
# Print word counts
# Get feature names to use as dataframe column headers
dtm2 = pd.DataFrame(dtm2.todense(), columns=tfidf.get_feature_names())
# View Feature Matrix as DataFrame
dtm2.head()

,$,+,+ year,2,3,5,\xe2\x80\x93,ability,ability work,able,...,workplace,world,world\xe2\x80\x99s,write,write communication,write verbal,ws,year,year experience,you\xe2\x80\x99ll
0,0.000000,0.000000,0.000000,0.136188,0.0,0.000000,0.00000,0.132288,0.0,0.0,...,0.0,0.000000,0.0,0.084484,0.0,0.0,0.000000,0.000000,0.0,0.0
1,0.000000,0.042720,0.046647,0.000000,0.0,0.000000,0.00000,0.033343,0.0,0.0,...,0.0,0.081134,0.0,0.085175,0.0,0.0,0.069542,0.030503,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.081452,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
3,0.365729,0.000000,0.000000,0.000000,0.0,0.085178,0.00000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.040626,0.0,0.0
4,0.000000,0.213765,0.233415,0.000000,0.0,0.000000,0.31155,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.152635,0.0,0.0


In [ ]:
wc = count(dtm2)
wc.head(20)

,word,appears_in,count,rank,pct_total,cul_pct_total,appears_in_pct
6,e,589,933,1.0,0.126320,0.126320,1.382629
16,i,439,644,2.0,0.087192,0.213512,1.030516
5,a,437,604,3.0,0.081776,0.295288,1.025822
18,t,444,588,4.0,0.079610,0.374898,1.042254
26,n,414,553,5.0,0.074871,0.449770,0.971831
4,r,407,483,6.0,0.065394,0.515164,0.955399
20,o,339,417,7.0,0.056458,0.571622,0.795775
17,l,333,391,8.0,0.052938,0.624560,0.781690
22,c,305,366,9.0,0.049553,0.674113,0.715962
25,s,266,352,10.0,0.047658,0.721771,0.624413


## 6) Create a NearestNeighbor Model. Write the description of your ideal datascience job and query your job listings. 

In [ ]:
##### Your Code Here #####
raise Exception("\nThis task is not complete. \nReplace this line with your code for the task."

## Stretch Goals

 - Try different visualizations for words and frequencies - what story do you want to tell with the data?
 - Scrape Job Listings for the job title "Data Analyst". How do these differ from Data Scientist Job Listings
 - Try and identify requirements for experience specific technologies that are asked for in the job listings. How are those distributed among the job listings?
 - Use a clustering algorithm to cluster documents by their most important terms. Do the clusters reveal any common themes?
  - **Hint:** K-means might not be the best algorithm for this. Do a little bit of research to see what might be good for this. Also, remember that algorithms that depend on Euclidean distance break down with high dimensional data.
 - Create a labeled dataset - which jobs will you apply for? Train a model to select the jobs you are most likely to apply for. :) 